In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
%tensorflow_version 1.x

In [ ]:
!pip install tiffile
!pip install elasticdeform
!pip install keras==2.2.5
!pip install csbdeep
!pip install stardist
!pip install gputools
!pip install btrack

In [ ]:
%cd '/content/drive/My Drive/BTrack/SmartSeeds/Utils/'
import os
import glob
import sys

from tifffile import imread, imwrite
import numpy as np
from tqdm import tqdm
from Augmentation import Augmentation3D
from helpers import Resizer3D
from pathlib import Path

In [ ]:
ImageDir =  '/content/drive/My Drive/BTrack/TrainingDatas/GreenCell3D/WinterPatchRaw/'
LabelDir = '/content/drive/My Drive/BTrack/TrainingDatas/GreenCell3D/WinterPatchMask/'

ResizeImageDir =  '/content/drive/My Drive/BTrack/TrainingDatas/GreenCell3D/WinterDenoisedRaw/'
ResizeLabelDir = '/content/drive/My Drive/BTrack/TrainingDatas/GreenCell3D/WinterRealMask/'

Resize = False

In [ ]:
if Resize:
        Raw_path = os.path.join(ImageDir, '*tif')
        filesRaw = glob.glob(Raw_path)
        filesRaw.sort

        Label_path = os.path.join(LabelDir, '*tif')
        filesLabel = glob.glob(Label_path)
        filesLabel.sort

        SizeX = 1024
        SizeY = 1024
        SizeZ = 65
        Data = []
        Label = []
        for fname in filesRaw:
            
            for secondfname in filesLabel:
                
                Name = os.path.basename(os.path.splitext(fname)[0])
                LabelName = os.path.basename(os.path.splitext(secondfname)[0])
                if Name == LabelName:
                    image = Resizer3D(imread(fname), SizeX, SizeY, SizeZ)
                    Data.append(image)
                    labelimage = Resizer3D(imread(secondfname), SizeX, SizeY, SizeZ)
                    Label.append(labelimage)   
            
        Data = np.asarray(Data)
        Label = np.asarray(Label)
        print(Data.shape, Label.shape)    
else:

        Raw_path = os.path.join(ResizeImageDir, '*tif')
        filesRaw = glob.glob(Raw_path)
        filesRaw.sort

        Label_path = os.path.join(ResizeLabelDir, '*tif')
        filesLabel = glob.glob(Label_path)
        filesLabel.sort

        Data = []
        Label = []
        for fname in filesRaw:
            
            for secondfname in filesLabel:
                
                Name = os.path.basename(os.path.splitext(fname)[0])
                LabelName = os.path.basename(os.path.splitext(secondfname)[0])
                if Name == LabelName:
                    image = imread(fname)
                    Data.append(image)
                    labelimage = imread(secondfname)
                    Label.append(labelimage)   
            
        Data = np.asarray(Data)
        Label = np.asarray(Label)
        print(Data.shape, Label.shape)   



In [ ]:
if Resize:

        Path(ResizeImageDir).mkdir(exist_ok=True)
        Path(ResizeLabelDir).mkdir(exist_ok=True)
        Count = 0
        for i in range(0, Data.shape[0]):
          Name = 'GreenCellsClaudia' + str(Count)
          imwrite(ResizeImageDir + '/' + Name + '.tif', Data[i].astype('float16'))
          imwrite(ResizeLabelDir + '/' + Name + '.tif', Label[i].astype('uint16'))
          Count = Count + 1

In [ ]:
AugmentImageDir =  '/content/drive/My Drive/BTrack/TrainingDatas/GreenCell3D/WinterDenoisedRaw/'
AugmentLabelDir = '/content/drive/My Drive/BTrack/TrainingDatas/GreenCell3D/WinterRealMask/'
Path(AugmentImageDir).mkdir(exist_ok=True)
Path(AugmentLabelDir).mkdir(exist_ok=True)

In [ ]:
#Shift the image and label pair along axis 1 (z axis)
AugmentData = Augmentation3D(shift_axis=1, shift_range = 1)
AugImageLabel = AugmentData.build(data=Data, label=Label, batch_size = Data.shape[0])



In [ ]:
#Save the shifts
AugPair = np.asarray(next(AugImageLabel))
Count = 0
for i in range(0, AugPair.shape[1]):
     Name = 'GreenCellsClaudiaShiftedPos' + str(Count)
     imwrite(AugmentImageDir + '/' + Name + '.tif', AugPair[0,i,:,:].astype('float32'))
     imwrite(AugmentLabelDir + '/' + Name + '.tif', AugPair[1,i,:,:].astype('uint16'))
     Count = Count + 1

In [ ]:
#Shift the image and label pair along axis 1 (z axis)
AugmentData = Augmentation3D(shift_axis=1, shift_range = -1)
AugImageLabel = AugmentData.build(data=Data, label=Label, batch_size = Data.shape[0])

In [ ]:
#Save the shifts
AugPair = np.asarray(next(AugImageLabel))
Count = 0
for i in range(0, AugPair.shape[1]):
     Name = 'GreenCellsClaudiaShiftedNeg' + str(Count)
     imwrite(AugmentImageDir + '/' + Name + '.tif', AugPair[0,i,:,:].astype('float32'))
     imwrite(AugmentLabelDir + '/' + Name + '.tif', AugPair[1,i,:,:].astype('uint16'))
     Count = Count + 1

In [ ]:
#Flip the image and label pair along axis 1 (z axis)
AugmentData = Augmentation3D(flip_axis=1)
AugImageLabel = AugmentData.build(data=Data, label=Label, batch_size = Data.shape[0])


In [ ]:
#Save the flips
AugPair = np.asarray(next(AugImageLabel))
Count = 0
for i in range(0, AugPair.shape[1]):
     Name = 'GreenCellsClaudiaFlip' + str(Count)
     imwrite(AugmentImageDir + '/' + Name + '.tif', AugPair[0,i,:,:].astype('float32'))
     imwrite(AugmentLabelDir + '/' + Name + '.tif', AugPair[1,i,:,:].astype('uint16'))
     Count = Count + 1